In [1]:
import sys
sys.path.append("../")
from data import Data
from matplotlib import pyplot as plt
import numpy as np
from fpl import FPL
import aiohttp
import asyncio
import pandas as pd
from random import shuffle
from collections import defaultdict
from scipy.stats import skewnorm
print("hello FPL")

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
hello FPL


In [2]:
## Get current match day squad
## Find player with the least expected score, swap him for player with highest expected score under budget
## Make player with highest expected score in squad as captain

In [3]:
async def get_current_squad():
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        await fpl.login(email=os.environ['email'], password=os.environ['password'])
        user = await fpl.get_user(5645003)
        squad = await user.get_team()
        position_map = {
        1: "Goalkeeper",
        2: "Defender",
        3: "Midfielder",
        4: "Forward"
        }
        current_squad = pd.DataFrame(columns=["name", "element", "selling_price", "purchase_price", "is_captain"])
        for i, player_element in enumerate(squad):
            player = await fpl.get_player(player_element['element'])
            name = player.first_name + " " + player.second_name
            current_squad.loc[i, 'name'] = name
            current_squad.loc[i, 'element'] = player_element['element']
            current_squad.loc[i, "selling_price"] = player_element["selling_price"]
            current_squad.loc[i, "purchase_price"] = player_element["purchase_price"]
            current_squad.loc[i, "is_captain"] = player_element["is_captain"]
        return current_squad

current_squad = await get_current_squad()
current_squad

,name,element,selling_price,purchase_price,is_captain
0,Hugo Lloris,383,55,55,False
1,Trent Alexander-Arnold,259,75,75,True
2,Willy Boly,458,55,55,False
3,Matt Doherty,457,60,60,False
4,Marcos Alonso,104,60,60,False
5,Willian Borges Da Silva,478,80,80,False
6,Declan Rice,448,50,50,False
7,Jonjo Shelvey,321,55,55,False
8,Matt Ritchie,322,50,50,False
9,Michail Antonio,437,65,65,False


In [4]:
data = Data()
await data.get_value_data()
data.build_player_timeseries()
recent_form_map = data.build_recent_form_map()
performance = data.predict_performance(recent_form_map)
value_data = data.value_data
performance_value = pd.merge(performance, value_data, on = ['name'])
performance_value.head()

,name,42,43,44,45,46,47,54,predicted_score,position,points_per_game,minutes,now_cost,value,element
0,Bruno Miguel Borges Fernandes,15.0,14.0,5.0,6.0,1.0,11.0,0.0,8.5,Midfielder,0.0,0.0,105.0,0.000000,302
1,Gabriel Fernando de Jesus,1.0,8.0,14.0,8.0,2.0,8.0,0.0,8,Forward,0.0,0.0,95.0,0.000000,282
2,Anthony Martial,14.0,6.0,13.0,8.0,5.0,6.0,0.0,7,Forward,0.0,0.0,90.0,0.000000,303
3,Michail Antonio,7.0,2.0,26.0,7.0,9.0,0.0,2.0,7,Forward,2.0,90.0,65.0,0.030769,437
4,Jamie Vardy,13.0,9.0,7.0,5.0,2.0,1.0,13.0,6,Forward,13.0,90.0,100.0,0.130000,224


In [5]:
player_dropped = pd.merge(current_squad, performance_value, on = ['name', 'element']).sort_values(by=['predicted_score'], ascending = True).head(1)
player_dropped

,name,element,selling_price,purchase_price,is_captain,42,43,44,45,46,47,54,predicted_score,position,points_per_game,minutes,now_cost,value
12,Benjamin Chilwell,232,55,55,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Defender,0.0,0.0,55.0,0.0


In [6]:
non_squad_players = performance_value[(~performance_value['name'].isin(current_squad['name'])) & (performance_value['now_cost'] <= player_dropped["now_cost"].values[0]) & (performance_value['position'] == player_dropped['position'].values[0])]
best_replacement_player = non_squad_players.sort_values(by=['predicted_score'], ascending = False).head(1)
best_replacement_player


,name,42,43,44,45,46,47,54,predicted_score,position,points_per_game,minutes,now_cost,value,element
5,Toby Alderweireld,7.0,6.0,10.0,1.0,6.0,1.0,2.0,6,Defender,2.0,90.0,55.0,0.036364,386


In [7]:
new_squad = pd.DataFrame(current_squad[current_squad['name'] != player_dropped['name'].values[0]])
new_squad.loc[len(new_squad), 'name'] = best_replacement_player['name'].values[0]
new_squad.loc[len(new_squad), 'element'] = best_replacement_player['element'].values[0]
new_squad.loc[len(new_squad), 'is_captain'] = False
new_squad.loc[len(new_squad), 'purchase_price'] = best_replacement_player['now_cost'].values[0]
new_squad.loc[len(new_squad), 'selling_price'] = None 
new_squad

,name,element,selling_price,purchase_price,is_captain
0,Hugo Lloris,383,55,55,False
1,Trent Alexander-Arnold,259,75,75,True
2,Willy Boly,458,55,55,False
3,Matt Doherty,457,60,60,False
4,Marcos Alonso,104,60,60,False
5,Willian Borges Da Silva,478,80,80,False
6,Declan Rice,448,50,50,False
7,Jonjo Shelvey,321,55,55,False
8,Matt Ritchie,322,50,50,False
9,Michail Antonio,437,65,65,False


In [13]:
new_squad_performance_value = pd.merge(new_squad, performance_value, on = ['name'])
new_squad_performance_value.sort_values(by=["position", 'predicted_score'], inplace = True, ascending = False)
new_squad_performance_value

,name,element_x,selling_price,purchase_price,is_captain,42,43,44,45,46,47,54,predicted_score,position,points_per_game,minutes,now_cost,value,element_y
12,Bruno Miguel Borges Fernandes,302,105,105,False,15.0,14.0,5.0,6.0,1.0,11.0,0.0,8.5,Midfielder,0.0,0.0,105.0,0.000000,302
6,Declan Rice,448,50,50,False,7.0,2.0,3.0,10.0,6.0,5.0,2.0,5.5,Midfielder,2.0,90.0,50.0,0.040000,448
5,Willian Borges Da Silva,478,80,80,False,9.0,11.0,2.0,3.0,4.0,0.0,14.0,3.5,Midfielder,14.0,74.0,81.0,0.172840,478
7,Jonjo Shelvey,321,55,55,False,10.0,2.0,2.0,1.0,3.0,5.0,3.0,2.5,Midfielder,3.0,90.0,55.0,0.054545,321
8,Matt Ritchie,322,50,50,False,0.0,0.0,2.0,9.0,5.0,1.0,0.0,1.5,Midfielder,0.0,0.0,50.0,0.000000,322
0,Hugo Lloris,383,55,55,False,10.0,6.0,3.0,3.0,10.0,2.0,3.0,4.5,Goalkeeper,3.0,90.0,55.0,0.054545,383
11,Nick Pope,96,55,55,False,2.0,8.0,3.0,3.0,6.0,1.0,0.0,3,Goalkeeper,0.0,0.0,55.0,0.000000,96
9,Michail Antonio,437,65,65,False,7.0,2.0,26.0,7.0,9.0,0.0,2.0,7,Forward,2.0,90.0,65.0,0.030769,437
10,Danny Ings,366,85,85,False,2.0,9.0,2.0,7.0,4.0,8.0,2.0,5.5,Forward,2.0,90.0,85.0,0.023529,366
13,Toby Alderweireld,386,None,55,False,7.0,6.0,10.0,1.0,6.0,1.0,2.0,6,Defender,2.0,90.0,55.0,0.036364,386


In [14]:
current_performance_value = pd.merge(current_squad, performance_value, on = ['name', 'element']).sort_values(by=['predicted_score', 'position'], ascending = False)
current_performance_value

,name,element,selling_price,purchase_price,is_captain,42,43,44,45,46,47,54,predicted_score,position,points_per_game,minutes,now_cost,value
13,Bruno Miguel Borges Fernandes,302,105,105,False,15.0,14.0,5.0,6.0,1.0,11.0,0.0,8.5,Midfielder,0.0,0.0,105.0,0.000000
9,Michail Antonio,437,65,65,False,7.0,2.0,26.0,7.0,9.0,0.0,2.0,7,Forward,2.0,90.0,65.0,0.030769
14,Anthony Martial,303,90,90,False,14.0,6.0,13.0,8.0,5.0,6.0,0.0,7,Forward,0.0,0.0,90.0,0.000000
6,Declan Rice,448,50,50,False,7.0,2.0,3.0,10.0,6.0,5.0,2.0,5.5,Midfielder,2.0,90.0,50.0,0.040000
10,Danny Ings,366,85,85,False,2.0,9.0,2.0,7.0,4.0,8.0,2.0,5.5,Forward,2.0,90.0,85.0,0.023529
0,Hugo Lloris,383,55,55,False,10.0,6.0,3.0,3.0,10.0,2.0,3.0,4.5,Goalkeeper,3.0,90.0,55.0,0.054545
5,Willian Borges Da Silva,478,80,80,False,9.0,11.0,2.0,3.0,4.0,0.0,14.0,3.5,Midfielder,14.0,74.0,81.0,0.172840
11,Nick Pope,96,55,55,False,2.0,8.0,3.0,3.0,6.0,1.0,0.0,3,Goalkeeper,0.0,0.0,55.0,0.000000
7,Jonjo Shelvey,321,55,55,False,10.0,2.0,2.0,1.0,3.0,5.0,3.0,2.5,Midfielder,3.0,90.0,55.0,0.054545
2,Willy Boly,458,55,55,False,1.0,2.0,6.0,2.0,6.0,1.0,6.0,2,Defender,6.0,90.0,55.0,0.109091
